In [46]:
import cv2
import os
import numpy as np

# Capture live video
cap = cv2.VideoCapture(0)

# Get current width of frame
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
# Get current height of frame
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*"XVID")
out = cv2.VideoWriter("output_video.mp4", fourcc, 30.0, (int(width), int(height)))

# Initialize reference background
ret, ref_bg = cap.read()
ref_bg = cv2.cvtColor(ref_bg, cv2.COLOR_BGR2GRAY)

unsafe_text_color = (0, 0, 255)  # Red color
thief_detected_text_color = (0, 255, 0)  # Green color

while cap.isOpened():
    ret, frame = cap.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    difference_frame = cv2.absdiff(gray, ref_bg)
    thresh_frame = cv2.threshold(difference_frame, 50, 70, cv2.THRESH_BINARY)[1]
    cnts, hierarchy = cv2.findContours(thresh_frame.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    unsafe_detected = False
    thief_detected = False
    
    if cnts:
        for contour in cnts:
            if cv2.contourArea(contour) < 500:
                continue
            (x, y, w, h) = cv2.boundingRect(contour)
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 3)
            unsafe_detected = True
            thief_detected = True
            
    # Display "UNSAFE" text if motion detected
    if unsafe_detected:
        cv2.putText(frame, 'UNSAFE', (50, 50), cv2.FONT_HERSHEY_TRIPLEX, 2, unsafe_text_color, 2)
    
    # Display "Thief Detected" text if thief detected
    if thief_detected:
        cv2.putText(frame, 'Thief Detected', (50, 100), cv2.FONT_HERSHEY_TRIPLEX, 2, thief_detected_text_color, 2)
        
    cv2.imshow('Face Video', frame)
    cv2.imshow('Difference Frame', difference_frame)
    cv2.imshow('Threshold', thresh_frame)

    out.write(frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
